In [1]:
import requests
import json
import os
import subprocess
import sys
import base64
import hashlib
from urllib.parse import urlparse
import subprocess
import json
import time
import os
import glob


RSA_SIZE = 2048


DEFAULT_CURVE = "secp384r1"
FAST_CURVE = "secp256r1"
SUPPORTED_CURVES = [DEFAULT_CURVE, FAST_CURVE]

DIGEST_SHA384 = "sha384"
DIGEST_SHA256 = "sha256"

RSA_SIZE = 2048
# CCF network node
server="https://127.0.0.1:8000"

num_users = 4
# Getting Network metrices
url = server + "/app/api/metrics"

try:
    response = requests.get(url, verify='./workspace/sandbox_common/service_cert.pem')

    print("Status Code:", response.status_code)
    print("\nResponse Headers:")
    for header, value in response.headers.items():
        print(f"{header}: {value}")

    print("\nResponse Body:")
    try:
        # Attempt to parse JSON and print it in an indented format
        response_json = response.json()
        print(json.dumps(response_json, indent=4))
    except ValueError:
        # If response is not JSON, print as plain text
        print(response.text)

except requests.exceptions.RequestException as e:
    print("Error making request:", e)

Status Code: 200

Response Headers:
content-length: 176
content-type: application/json

Response Body:
{
    "metrics": [
        {
            "calls": 1,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "api/metrics",
            "retries": 0
        },
        {
            "calls": 2,
            "errors": 0,
            "failures": 0,
            "method": "GET",
            "path": "commit",
            "retries": 0
        }
    ]
}


In [2]:
%pip install matplotlib
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.0:
      Successfully uninstalled typing_extensions-4.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azure-core 1.29.7 requires typing-extensions>=4.6.0, but you have typing-extensions 4.5.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import requests
import json
import base64
import time
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.utils import to_categorical

server = "https://127.0.0.1:8000"  # Replace with your server URL



def download_global_model_weights(user_cert, user_key, model_id):
    try:
        print("Downloading client weights for aggregation...")
        response = requests.get(
            url=f"{server}/app/model/download_gloabl_weights?model_id={model_id}",
            verify="./workspace/sandbox_common/service_cert.pem",
            cert=(user_cert, user_key)
        )
        if response.status_code == 200:
            print("Client weights downloaded successfully.")
            response_data = json.loads(response.text)
            global_model_value = response_data.get("global_model")
            return global_model_value
    

            # # Access the value associated with the "gloablmodel" key
            # global_model_value = response_data.get("global_model")
            # model_id = global_model_value.get("model_id")
            # print(f"Global model ID: {model_id}")
            # return global_model_value
            

        else:
            print(f"Failed to download client weights. Status code: {response.status_code}")

        return None
    except Exception as e:
        print(f"An error occurred while downloading global model weights: {e}")
        return None
def aggregate_weight(client_weights_list):
    if client_weights_list:
        total_weights = sum(client_weights_list, [])
        aggregated_weights = [weight / len(client_weights_list) for weight in total_weights]
        return aggregated_weights
    else:
        return []
def create_model():
    print("Initializing the global model...")
    model = Sequential([
        Conv2D(64, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(32, kernel_size=3, activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def aggregate_weights(model_id,round_no, user_cert, user_key):
    print("Aggregating weights for model:", model_id)
  
    response = requests.put(
        url=f"{server}/app/model/aggregate_weights_local?model_id={model_id}&&round_no={round_no}",
        verify="./workspace/sandbox_common/service_cert.pem",
        cert=(user_cert, user_key)
    )
    print(response.text)
    if response.status_code == 200:
        print("Aggregation successful for model:", model_id)
    else:
        raise Exception(f"Failed to aggregate weights. Status code: {response.status_code}")
def train_model(model, X_train, y_train, X_test, y_test, user_id, round_no, epochs=2):
    print(f"Training model for User {user_id}, Round {round_no}...")
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)
    return history.history['loss']

def serialize_model(model):
    print("Serializing the model...")
    model.save('temp_model.h5')
    with open('temp_model.h5', 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')
    

def upload_initial_model(model_base64, user_cert, user_key):
    print("Uploading initial global model...")
    payload = {
      "global_model": {
        "model_name": "CNNModel",
        "model_data": model_base64
      }
    }
    response = requests.post(
        url=f"{server}/app/model/intial_model",
        verify="./workspace/sandbox_common/service_cert.pem",
        cert=(user_cert, user_key),
        json=payload
    )

    if response.status_code == 200:
        model_data = response.json()
        model_id = model_data.get("model_id")
        model_name = model_data.get("model_name")
        print(f"Initial global model '{model_name}' (ID: {model_id}) uploaded successfully.")
        return model_id
    else:
        print(f"Failed to upload initial model. Status code: {response.status_code}")
        return None
def deserialize_weights(serialized_weights):
    """ Deserialize serialized weights and set them to the model """
    print("Deserializing model weights...")
    
    # Parse the JSON-formatted string to obtain serialized weights
    deserialized_weights = json.loads(serialized_weights)
    
    # Convert the serialized weights back to numpy arrays
    value = [np.array(w) if isinstance(w, list) else w for w in deserialized_weights]
    
    # Set the deserialized weights to the model
    print("Model weights deserialized and set successfully.")
    return value
    

def serialize_weights(model):
    """ Serialize only the weights of the model """
    print("Serializing model weights...")
    weights = model.get_weights()  # Get the model weights as a list of numpy arrays
    #  print the shape of the weights
    # print("Weights shape: before serialization")
    # for w in weights:
    #     print(w.shape)
    # Convert the weights to a JSON-serializable format
    serialized_weights = [w.tolist() if isinstance(w, np.ndarray) else w for w in weights]
    # print the shape of the serialized weights
    print("Weights shape: after serialization")
    # for w in serialized_weights:
    #     print(len(w))
    
    return json.dumps(serialized_weights)


def upload_model_weights(model_weights_base64, user_cert, user_key, round_no, model_id=None):
    """ Upload only the model weights """
    print(f"Uploading model weights for Round {round_no}...")
    payload = {
        "model_id": model_id,
        "weights_json": model_weights_base64,
        "round_no": round_no
    }
    response = requests.post(
        url=f"{server}/app/model/upload/local_model_weights",
        verify="./workspace/sandbox_common/service_cert.pem",
        cert=(user_cert, user_key),
        json=payload
    )
    print(response.text)
    if response:
        print(f"Model weights uploaded successfully for Round {round_no}.")
    else:
        raise Exception(f"Failed to upload model weights. Status code: {response.status_code}")

def download_model(user_cert, user_key, user_id, round_no, max_retries=5, model_id=None):
    attempts = 0
    while attempts < max_retries:
        print(f"Attempting to download model for User {user_id}, Round {round_no}, Attempt {attempts + 1}...")
        response = requests.get(
            url=f"{server}/app/model/download/global?model_id={model_id}",
            verify="./workspace/sandbox_common/service_cert.pem",
            cert=(user_cert, user_key)
        )
  

        if response.status_code == 200:
            model_data = response.json().get("model_details", {})
            model_base64 = model_data
            
            if model_base64:
                with open('temp_model.h5', 'wb') as file:
                    file.write(base64.b64decode(model_base64))
                return load_model('temp_model.h5')
            else:
                print("Model data not found in response, retrying...")
        else:
            print(f"Failed to download model. Status code: {response.status_code}, retrying...")
        
        time.sleep(2)  # Delay before retrying
        attempts += 1
    
    raise Exception("Failed to download model after maximum retries.")

def plot_loss(user_losses):
    for user_id, losses in user_losses.items():
        plt.plot(losses, label=f'User {user_id}')
    plt.title('Model Loss per Training Round')
    plt.ylabel('Loss')
    plt.xlabel('Round')
    plt.legend()
    plt.show()


2024-01-28 23:20:33.698552: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 23:20:33.751680: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-28 23:20:33.753260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-28 23:20:34.594783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Load and preprocess MNIST dataset
print("Loading and preprocessing MNIST dataset...")
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Split the dataset for two users
print("Splitting dataset for two users...")
split_index = int(len(X_train) / 2)
X_train_user0, X_train_user1 = np.split(X_train, [split_index])
y_train_user0, y_train_user1 = np.split(y_train, [split_index])

# Initialize and train the global model (User 0)
global_model = create_model()
train_model(global_model, X_train_user0, y_train_user0, X_test, y_test, user_id=0, round_no=0)


Loading and preprocessing MNIST dataset...
11490434/11490434 [==============================] - 0s 0us/step
Splitting dataset for two users...
Initializing the global model...
Training model for User 0, Round 0...
Epoch 1/2
938/938 [==============================] - 14s 14ms/step - loss: 0.6313 - accuracy: 0.8975 - val_loss: 0.1492 - val_accuracy: 0.9551
Epoch 2/2
938/938 [==============================] - 13s 14ms/step - loss: 0.1169 - accuracy: 0.9661 - val_loss: 0.0995 - val_accuracy: 0.9710


[0.6313133239746094, 0.1168571338057518]

In [5]:
# Serialize and upload initial global model
model_base64 = serialize_model(global_model)

initial_model_id = upload_initial_model(model_base64, "./workspace/sandbox_common/user0_cert.pem", "./workspace/sandbox_common/user0_privk.pem")

Serializing the model...
Uploading initial global model...
Initial global model 'CNNModel' (ID: 0) uploaded successfully.


/workspaces/ccf-app-template/.venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:

if initial_model_id is None:
    raise Exception("Failed to upload the initial global model. Stopping the process.")

# Wait for some time after initial upload
# time.sleep(1)  # Wait for 10 seconds

# Federated Learning Process
# Federated Learning Process
num_rounds = 4  # Number of training rounds
user_losses = {0: [], 1: []}  # To track losses for each user

# Download the latest global model once for each client
global_model = download_model(
    "./workspace/sandbox_common/user0_cert.pem",
    "./workspace/sandbox_common/user0_privk.pem",
    user_id=0, round_no=0, model_id=initial_model_id
)

# Initialize the local models for both users
local_model_user0 = global_model
local_model_user1 = global_model

for round_no in range(1, num_rounds + 1):
    for user_id in range(2):  # Two users: 0 and 1
        # Train the local model on the user's data
        X_train_user = X_train_user0 if user_id == 0 else X_train_user1
        y_train_user = y_train_user0 if user_id == 0 else y_train_user1
        loss = train_model(local_model_user0 if user_id == 0 else local_model_user1,
                           X_train_user, y_train_user, X_test, y_test, user_id, round_no)
        user_losses[user_id].extend(loss)

        # Serialize and upload the updated model weights for the current round
        model_weights_base64 = serialize_weights(local_model_user0 if user_id == 0 else local_model_user1)
        # Upload model weights only if they are not empty

        if model_weights_base64:
            upload_model_weights(model_weights_base64, f"./workspace/sandbox_common/user{user_id}_cert.pem",
                                 f"./workspace/sandbox_common/user{user_id}_privk.pem", round_no, model_id=initial_model_id)
        else:
            print("Model weights are empty, skipping upload...")

    # Aggregate weights after each user's training round
    aggregate_weights(initial_model_id,round_no, "./workspace/sandbox_common/member0_cert.pem",
                      "./workspace/sandbox_common/member0_privk.pem")

    # Download global model weights after aggregation and update local models
    global_model_weights = download_global_model_weights(
        "./workspace/sandbox_common/user0_cert.pem", "./workspace/sandbox_common/user0_privk.pem",
        model_id=initial_model_id
    )

    # if global_model_weights:
    #     # Update local models with global weights
    #     local_model_user0.set_weights([np.array(w) for w in global_model_weights])
    #     local_model_user1.set_weights([np.array(w) for w in global_model_weights])
    #     print("Global model weights updated for both clients.")

print("Federated Learning Process Completed.")

# Plot loss graphs for each user
plot_loss(user_losses)


Attempting to download model for User 0, Round 0, Attempt 1...
Training model for User 0, Round 1...
Epoch 1/2
938/938 [==============================] - 13s 13ms/step - loss: 0.0826 - accuracy: 0.9760 - val_loss: 0.0933 - val_accuracy: 0.9714
Epoch 2/2
938/938 [==============================] - 13s 14ms/step - loss: 0.0672 - accuracy: 0.9801 - val_loss: 0.0848 - val_accuracy: 0.9757
Serializing model weights...
Weights shape: after serialization
Uploading model weights for Round 1...
{"message":"Model weights received"}
Model weights uploaded successfully for Round 1.
Training model for User 1, Round 1...
Epoch 1/2
938/938 [==============================] - 12s 13ms/step - loss: 0.0842 - accuracy: 0.9748 - val_loss: 0.0672 - val_accuracy: 0.9786
Epoch 2/2
938/938 [==============================] - 12s 13ms/step - loss: 0.0554 - accuracy: 0.9832 - val_loss: 0.0657 - val_accuracy: 0.9823
Serializing model weights...
Weights shape: after serialization
Uploading model weights for Round 1.

In [65]:
import json

# Read the serialized weights from the test.txt file
with open('test.txt', 'r') as file:
    serialized_weights = file.read()

# Deserialize the weights
deserialized_weights = json.loads(serialized_weights)

# Assuming 'model' is your Keras model
# Set the weights of the model
deserialize_weights


JSONDecodeError: Extra data: line 2 column 1 (char 557311)